# Hospital Forecasting Pipeline: Architecture & Experimental Design

## Objective
Develop a generalizable hierarchical forecasting system for hospital capacity planning that integrates classical baselines with machine learning approaches while maintaining temporal integrity and handling heterogeneous facility scales.

## Datasets & Temporal Configuration
- **Hospital Dataset** (primary): monthly aggregates, H∈{6,12} months
- **Tourism & Electricity** (auxiliary): exploratory benchmarks

## Core Principles
1. **Temporal Leakage Prevention**: rolling-origin backtest with strict train/test cutoffs
2. **Bias-Variance Tradeoff Management**: segmentation by facility volume to reduce variance across heterogeneous scales
3. **Cold-Start Handling**: minimum training window (48 months) ensures stable feature estimation
4. **Residual Analysis**: compute and track error distributions for model selection

In [1]:
import os

# Enumerate all available time-series files in the workspace
tsf_files = [f for f in os.listdir('.') if f.lower().endswith('.tsf')]
tsf_files.sort()

print(f"Found {len(tsf_files)} .tsf files:\n")
for f in tsf_files:
    print(f)

Found 3 .tsf files:

electricity_weekly_dataset.tsf
hospital_dataset.tsf
tourism_monthly_dataset.tsf


## Cell 2: Metadata Inspection & Format Validation

### Technical Rationale
The Monash Time-Series Format (TSF) is a human-readable interchange format for heterogeneous forecasting datasets. Before ingesting 50k+ series rows, we validate:
- Metadata consistency (frequency, horizon, missing-value indicator)
- Attribute schema alignment
- Numeric parsing robustness on a sample (prevents downstream surprises)

This lightweight inspection runs once and prevents re-scanning the entire file during feature engineering.

In [2]:
import os
import re
from pathlib import Path
from collections import defaultdict

TSF_FILES = [
    "hospital_dataset.tsf",
    "tourism_monthly_dataset.tsf",
    "electricity_weekly_dataset.tsf",
]

def parse_tsf(filepath, max_series_preview=3):
    """
    Stream-parse TSF metadata and sample data without loading entire file into memory.
    Returns schema (attributes), temporal config (@frequency, @horizon), and series length distribution.
    """
    meta = {}
    attributes = []
    header_comments = []
    in_data = False
    series_lengths = []
    series_names_preview = []
    start_timestamps_preview = []
    numeric_parse_failures = 0

    attr_pat = re.compile(r"^@attribute\s+(\S+)\s+(\S+)", re.IGNORECASE)

    with open(filepath, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            if line.startswith("#") and not in_data:
                header_comments.append(line)
                continue

            if line.lower().startswith("@data"):
                in_data = True
                continue

            if not in_data:
                if line.lower().startswith("@attribute"):
                    m = attr_pat.match(line)
                    if m:
                        attributes.append((m.group(1), m.group(2)))
                    continue

                if line.startswith("@"):
                    parts = line.split(None, 1)
                    key = parts[0].lstrip("@").strip().lower()
                    val = parts[1].strip() if len(parts) > 1 else ""
                    meta[key] = val
                continue

            # Parse data row: series_name:start_timestamp:val1,val2,val3,...
            if in_data:
                if len(series_lengths) >= 5000 and max_series_preview == 0:
                    break

                parts = line.split(":", 2)
                if len(parts) < 2:
                    continue

                sname = parts[0]
                series_names_preview.append(sname) if len(series_names_preview) < max_series_preview else None

                if len(parts) == 2:
                    values_str = parts[1]
                else:
                    ts = parts[1]
                    values_str = parts[2]
                    if len(start_timestamps_preview) < max_series_preview:
                        start_timestamps_preview.append(ts)

                values = [v.strip() for v in values_str.split(",") if v.strip() != ""]
                series_lengths.append(len(values))

                # Validate numeric parsability on sample
                for v in values[:20]:
                    try:
                        float(v)
                    except:
                        numeric_parse_failures += 1

    summary = {
        "file": os.path.basename(filepath),
        "meta": meta,
        "attributes": attributes,
        "header_comment_lines": len(header_comments),
        "series_count_estimate": len(series_lengths),
        "series_length_min": min(series_lengths) if series_lengths else None,
        "series_length_median": (sorted(series_lengths)[len(series_lengths)//2] if series_lengths else None),
        "series_length_max": max(series_lengths) if series_lengths else None,
        "numeric_parse_failures_in_sample": numeric_parse_failures,
        "series_name_preview": series_names_preview[:max_series_preview],
        "start_timestamp_preview": start_timestamps_preview[:max_series_preview],
    }
    return summary

def print_summary(s):
    print("=" * 90)
    print(f"FILE: {s['file']}")
    print("-" * 90)

    for k in ["relation", "frequency", "horizon", "missing", "equallength"]:
        if k in s["meta"]:
            print(f"@{k}: {s['meta'][k]}")
    other_keys = [k for k in s["meta"].keys() if k not in {"relation","frequency","horizon","missing","equallength"}]
    if other_keys:
        print("Other @meta:")
        for k in sorted(other_keys):
            print(f"  @{k}: {s['meta'][k]}")

    print("\n@attributes:")
    if s["attributes"]:
        for name, typ in s["attributes"]:
            print(f"  - {name} ({typ})")
    else:
        print("  (none found)")

    print("\nData scan (limited):")
    print(f"  Series rows scanned: {s['series_count_estimate']}")
    print(f"  Series length min/median/max: {s['series_length_min']} / {s['series_length_median']} / {s['series_length_max']}")
    print(f"  Numeric parse failures (sampled values): {s['numeric_parse_failures_in_sample']}")

    print("\nPreview:")
    for i, name in enumerate(s["series_name_preview"]):
        ts = s["start_timestamp_preview"][i] if i < len(s["start_timestamp_preview"]) else "(no timestamp field)"
        print(f"  - series_name={name} | start_timestamp={ts}")

# Execute metadata inspection across all configured datasets
for fn in TSF_FILES:
    path = Path("./") / fn
    if not path.exists():
        print(f"[MISSING] {fn} not found in current directory.")
        continue
    summary = parse_tsf(path, max_series_preview=3)
    print_summary(summary)

FILE: hospital_dataset.tsf
------------------------------------------------------------------------------------------
@relation: Hospital
@frequency: monthly
@missing: false
@equallength: true

@attributes:
  - series_name (string)
  - start_timestamp (date)

Data scan (limited):
  Series rows scanned: 767
  Series length min/median/max: 84 / 84 / 84
  Numeric parse failures (sampled values): 0

Preview:
  - series_name=T1 | start_timestamp=2000-01-01 00-00-00
  - series_name=T2 | start_timestamp=2000-01-01 00-00-00
  - series_name=T3 | start_timestamp=2000-01-01 00-00-00
FILE: tourism_monthly_dataset.tsf
------------------------------------------------------------------------------------------
@relation: Tourism
@frequency: monthly
@horizon: 24
@missing: false
@equallength: false

@attributes:
  - series_name (string)
  - start_timestamp (date)

Data scan (limited):
  Series rows scanned: 366
  Series length min/median/max: 91 / 330 / 333
  Numeric parse failures (sampled values): 0



## Cell 3: Temporal Data Ingestion & Frequency Alignment

### Technical Rationale
TSF format encodes series as text; we convert to a normalized long-form DataFrame with:
- **Timezone Consistency**: convert all timestamps to UTC-naive to prevent comparison errors during rolling backtest
- **Frequency-Aware Interpolation**: respect declared frequency (monthly, weekly) when reconstructing timestamps from sparse start_ts
- **Missing-Value Handling**: NaN propagation through numeric coercion (pandas' `to_numeric(..., coerce=True)`)

This ensures that downstream feature engineering (lags, rolling statistics) aligns correctly with calendar boundaries.

In [3]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import re

TSF_FILES = [
    "hospital_dataset.tsf",
    "tourism_monthly_dataset.tsf",
    "electricity_weekly_dataset.tsf",
]

def parse_tsf_to_long_df(path: str) -> pd.DataFrame:
    """
    Parse TSF file into normalized long-form DataFrame.
    Handles missing timestamps gracefully and aligns to declared frequency.
    """
    meta = {}
    in_data = False
    rows = []

    with open(path, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue

            if line.lower().startswith("@data"):
                in_data = True
                continue

            if not in_data:
                if line.startswith("@"):
                    parts = line.split(None, 1)
                    key = parts[0].lstrip("@").strip().lower()
                    val = parts[1].strip() if len(parts) > 1 else ""
                    meta[key] = val
                continue

            # Parse data: series_name:start_timestamp:val1,val2,...
            parts = line.split(":", 2)
            if len(parts) < 2:
                continue

            series_name = parts[0]
            if len(parts) == 2:
                start_ts = None
                values_str = parts[1]
            else:
                start_ts = parts[1]
                values_str = parts[2]

            values = [v.strip() for v in values_str.split(",") if v.strip() != ""]
            y = pd.to_numeric(pd.Series(values), errors="coerce")

            freq = meta.get("frequency", "").lower()
            if start_ts is None:
                # Fallback: index-based pseudo-timestamps
                t = pd.RangeIndex(len(y))
            else:
                # Parse TSF timestamp format (may be "00-00-00" or ISO)
                start = pd.to_datetime(start_ts.replace("-", ":"), errors="coerce")
                if pd.isna(start):
                    start = pd.to_datetime(start_ts, errors="coerce")

                # Construct frequency-aligned date range
                if freq == "monthly":
                    t = pd.date_range(start=start, periods=len(y), freq="MS")
                elif freq == "weekly":
                    t = pd.date_range(start=start, periods=len(y), freq="W-SUN")
                else:
                    # Generic fallback for unlabeled frequencies
                    t = pd.date_range(start=start, periods=len(y), freq="D")

            df_part = pd.DataFrame({
                "dataset": Path(path).name,
                "series_name": series_name,
                "timestamp": t,
                "y": y.values
            })
            rows.append(df_part)

    out = pd.concat(rows, ignore_index=True)
    return out

# Ingest all configured datasets
dfs = {}
for fn in TSF_FILES:
    df = parse_tsf_to_long_df(fn)
    dfs[fn] = df
    print("="*80)
    print(fn)
    print("shape:", df.shape)
    print("columns:", df.columns.tolist())
    print("n_series:", df["series_name"].nunique())
    print("date_range:", df["timestamp"].min(), "->", df["timestamp"].max())
    print("missing_y:", df["y"].isna().sum())
    print(df.head(3))

hospital_dataset.tsf
shape: (64428, 4)
columns: ['dataset', 'series_name', 'timestamp', 'y']
n_series: 767
date_range: 2000-01-01 00:00:00+00:00 -> 2006-12-01 00:00:00+00:00
missing_y: 0
                dataset series_name                 timestamp   y
0  hospital_dataset.tsf          T1 2000-01-01 00:00:00+00:00  27
1  hospital_dataset.tsf          T1 2000-02-01 00:00:00+00:00  16
2  hospital_dataset.tsf          T1 2000-03-01 00:00:00+00:00  18
tourism_monthly_dataset.tsf
shape: (109280, 4)
columns: ['dataset', 'series_name', 'timestamp', 'y']
n_series: 366
date_range: 1979-01-01 00:00:00+00:00 -> 2007-09-01 00:00:00+00:00
missing_y: 0
                       dataset series_name                 timestamp  \
0  tourism_monthly_dataset.tsf          T1 1979-01-01 00:00:00+00:00   
1  tourism_monthly_dataset.tsf          T1 1979-02-01 00:00:00+00:00   
2  tourism_monthly_dataset.tsf          T1 1979-03-01 00:00:00+00:00   

           y  
0  1149.8700  
1  1053.8002  
2  1388.8798  
elect

## Cell 4: Rolling-Origin Backtesting Strategy

### Critical Design: Temporal Leakage Prevention
We implement a **rolling-origin window** per series to simulate realistic deployment conditions:
- **Hard Cutoff**: `train_end < test_start` always holds (strict leakage check)
- **Minimum Training Window**: 48 months (hospital domain: need ≥4 years to capture seasonal + trend variance)
- **Variable Step Size**: step=3 months (hospital) allows overlapping windows without temporal overlap
- **Horizon Coverage**: H={6,12} months reflects planning cycles (quarterly + annual)

This avoids the "look-ahead bias" endemic in time-series ML: training on future values pretending they're past.

### Leakage Verification
We compute `(splits_df["train_end"] < splits_df["test_start"]).all()` to ensure the backtest is valid before model training.

In [4]:
import pandas as pd
from pandas.tseries.frequencies import to_offset

# Configuration locked for reproducibility
DATASETS = {
    "hospital_dataset.tsf": {
        "freq": "MS",          # Monthly start
        "horizons": [6, 12],   # Quarterly and annual planning cycles
        "min_train_periods": 48,  # Minimum 4 years to stabilize seasonal estimation
        "step": 3              # Step between rolling windows (months)
    },
    "tourism_monthly_dataset.tsf": {
        "freq": "MS",
        "horizons": [24],
        "min_train_periods": 96,  # 8 years for unequal-length series
        "step": 6
    },
    "electricity_weekly_dataset.tsf": {
        "freq": "W-SUN",
        "horizons": [8],
        "min_train_periods": 104,  # 2 years of weekly data
        "step": 4
    }
}

def build_rolling_splits(df, dataset_name, cfg):
    """
    Construct rolling-origin splits per series (one series = one independent backtest).
    Returns DataFrame with fields:
      series_name | split_id | train_end | test_start | test_end | horizon
    """
    rows = []
    freq = cfg["freq"]
    step = cfg["step"]
    min_train = cfg["min_train_periods"]

    for horizon in cfg["horizons"]:
        for s, g in df[df["dataset"] == dataset_name].groupby("series_name"):
            g = g.sort_values("timestamp")
            times = g["timestamp"].values
            n = len(times)

            # Compute valid cutoff indices (must have min_train data before cutoff)
            cutoff_idxs = list(range(min_train - 1, n - horizon, step))
            for i, cut_idx in enumerate(cutoff_idxs):
                train_end = times[cut_idx]
                test_start = times[cut_idx + 1]
                test_end = times[cut_idx + horizon]

                rows.append({
                    "dataset": dataset_name,
                    "series_name": s,
                    "horizon": horizon,
                    "split_id": i,
                    "train_end": pd.Timestamp(train_end),
                    "test_start": pd.Timestamp(test_start),
                    "test_end": pd.Timestamp(test_end)
                })

    return pd.DataFrame(rows)

# Generate rolling splits for all configured datasets
all_splits = []
for name, cfg in DATASETS.items():
    df = dfs[name]
    splits = build_rolling_splits(df, name, cfg)
    all_splits.append(splits)

splits_df = pd.concat(all_splits, ignore_index=True)

# Verification: aggregate splits and temporal coverage
summary = (
    splits_df
    .groupby(["dataset", "horizon"])
    .agg(
        n_splits=("split_id", "nunique"),
        n_series=("series_name", "nunique"),
        first_train_end=("train_end", "min"),
        last_test_end=("test_end", "max")
    )
    .reset_index()
)

print("="*90)
print("ROLLING BACKTEST SUMMARY")
print("="*90)
print(summary)

# **HARD LEAKAGE CHECK**: verify no train/test contamination
leakage_check = (splits_df["train_end"] < splits_df["test_start"]).all()
print(f"\n✓ Leakage check (train_end < test_start for all splits): {leakage_check}")
if not leakage_check:
    raise RuntimeError("LEAKAGE DETECTED: train_end >= test_start for some split!")

print("\nSample splits (first 5):")
print(splits_df.head(5))

ROLLING BACKTEST SUMMARY
                          dataset  horizon  n_splits  n_series  \
0  electricity_weekly_dataset.tsf        8        12       321   
1            hospital_dataset.tsf        6        11       767   
2            hospital_dataset.tsf       12         9       767   
3     tourism_monthly_dataset.tsf       24        36       365   

  first_train_end last_test_end  
0      2013-12-22    2014-12-21  
1      2003-12-01    2006-12-01  
2      2003-12-01    2006-12-01  
3      1986-12-01    2007-06-01  

✓ Leakage check (train_end < test_start for all splits): True

Sample splits (first 5):
                dataset series_name  horizon  split_id  train_end test_start  \
0  hospital_dataset.tsf          T1        6         0 2003-12-01 2004-01-01   
1  hospital_dataset.tsf          T1        6         1 2004-03-01 2004-04-01   
2  hospital_dataset.tsf          T1        6         2 2004-06-01 2004-07-01   
3  hospital_dataset.tsf          T1        6         3 2004-09-01

## Cell 5: Baseline Seasonal Naive & Timezone Harmonization

### Rationale for Baseline
Seasonal naive (repeat last seasonal cycle) is the industry workhorse for hospital forecasting:
- Low computational cost (no optimization)
- Captures recurrent patterns without trend assumption
- Provides a reference point: if ML models don't beat this, they lack generalization

### Why Timezone Handling Matters
pandas may introduce timezone-aware timestamps when parsing dates. Subsequent `.dt.tz_localize(None)` ensures consistency:
- Avoids `TypeError` when comparing tz-aware vs. tz-naive in splits
- Simplifies groupby logic downstream

### Metric Design
- **MAE** (Mean Absolute Error): robust to outliers, interpretable in original units
- **RMSE** (Root Mean Squared Error): penalizes large forecast errors more heavily (captures tail risk)

In [5]:
import numpy as np
import pandas as pd

# Harmonize timezone representation across all dataframes (critical for merge/comparison)
def make_tz_naive(ts: pd.Series) -> pd.Series:
    ts = pd.to_datetime(ts, errors="coerce")
    try:
        return ts.dt.tz_localize(None)
    except AttributeError:
        return ts.dt.tz_localize(None)

for k in list(dfs.keys()):
    dfs[k]["timestamp"] = pd.to_datetime(dfs[k]["timestamp"], errors="coerce")
    if getattr(dfs[k]["timestamp"].dt, "tz", None) is not None:
        dfs[k]["timestamp"] = dfs[k]["timestamp"].dt.tz_localize(None)

for col in ["train_end", "test_start", "test_end"]:
    splits_df[col] = pd.to_datetime(splits_df[col], errors="coerce")
    if getattr(splits_df[col].dt, "tz", None) is not None:
        splits_df[col] = splits_df[col].dt.tz_localize(None)

# Seasonal naive baseline: repeats last observed seasonal cycle
def seasonal_naive_forecast(y_train: np.ndarray, horizon: int, season_lag: int) -> np.ndarray:
    """
    Seasonal naive forecast repeats the most recent seasonal cycle.
    For h > season_lag, patterns repeat cyclically.
    """
    y_train = np.asarray(y_train, dtype=float)
    if len(y_train) == 0:
        return np.full(horizon, np.nan)

    if len(y_train) < season_lag:
        # Cold-start: insufficient history; fallback to last observed value
        return np.repeat(y_train[-1], horizon)

    last_season = y_train[-season_lag:]
    reps = int(np.ceil(horizon / season_lag))
    y_hat = np.tile(last_season, reps)[:horizon]
    return y_hat

def evaluate_seasonal_naive(df_long, splits_df, dataset_name, freq):
    """
    Execute rolling-origin evaluation for seasonal naive baseline.
    Returns per-split metrics (MAE, RMSE).
    """
    season_lag = 12 if freq == "monthly" else 52  # 52 weeks per year
    df_ds = df_long[df_long["dataset"] == dataset_name]

    rows = []
    for _, s in splits_df[splits_df["dataset"] == dataset_name].iterrows():
        g = df_ds[df_ds["series_name"] == s["series_name"]].sort_values("timestamp")

        train = g[g["timestamp"] <= s["train_end"]]
        test  = g[(g["timestamp"] >= s["test_start"]) & (g["timestamp"] <= s["test_end"])]

        y_true = test["y"].astype(float).values
        y_hat  = seasonal_naive_forecast(train["y"].astype(float).values, int(s["horizon"]), season_lag)

        if len(y_true) != len(y_hat):
            continue  # Skip malformed split

        rows.append({
            "dataset": dataset_name,
            "horizon": int(s["horizon"]),
            "series_name": s["series_name"],
            "split_id": int(s["split_id"]),
            "mae": float(np.mean(np.abs(y_true - y_hat))),
            "rmse": float(np.sqrt(np.mean((y_true - y_hat) ** 2))),
        })

    return pd.DataFrame(rows)

# Evaluate baseline across all datasets
baseline_df = pd.concat([
    evaluate_seasonal_naive(dfs["hospital_dataset.tsf"], splits_df, "hospital_dataset.tsf", freq="monthly"),
    evaluate_seasonal_naive(dfs["tourism_monthly_dataset.tsf"], splits_df, "tourism_monthly_dataset.tsf", freq="monthly"),
    evaluate_seasonal_naive(dfs["electricity_weekly_dataset.tsf"], splits_df, "electricity_weekly_dataset.tsf", freq="weekly"),
], ignore_index=True)

summary = (
    baseline_df
    .groupby(["dataset", "horizon"], as_index=False)
    .agg(MAE=("mae", "mean"), RMSE=("rmse", "mean"))
)

print("="*90)
print("SEASONAL NAIVE BASELINE RESULTS")
print("="*90)
print(summary)
print(f"\nRows evaluated: {len(baseline_df)}")
print(f"Any NaNs in metrics: {baseline_df[['mae', 'rmse']].isna().any().to_dict()}")

SEASONAL NAIVE BASELINE RESULTS
                          dataset  horizon           MAE          RMSE
0  electricity_weekly_dataset.tsf        8  33504.051629  39984.900267
1            hospital_dataset.tsf        6     21.819545     26.014796
2            hospital_dataset.tsf       12     21.788582     27.021620
3     tourism_monthly_dataset.tsf       24   2006.442369   2521.121274

Rows evaluated: 30358
Any NaNs in metrics: {'mae': False, 'rmse': False}


## Cells 6–9: Classical Forecasting (ETS) & Rapid Prototyping

### Exponential Smoothing (Holt-Winters)
ETS provides a probabilistic alternative to neural networks:
- **Trend Component** (additive): captures linear drift in hospital demand
- **Seasonal Component** (additive): models recurring patterns
- **Parallel Batch Evaluation**: joblib enables horizontal scaling across 50+ series

### Rapid Global ML Prototype
Before segmentation, we train an **unconstrained global LightGBM** to understand baseline ML performance:
- Aggregates all series into one training matrix
- Quantile loss (α=0.5 = median) provides robustness
- Establishes whether ML > classical is worth the engineering cost

In [6]:
import os
os.environ["LIGHTGBM_VERBOSE"] = "-1"

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error

DATASET = "hospital_dataset.tsf"
HORIZONS = [6, 12]
LAGS = [1, 3, 6, 12]  # Tap into recent signal + seasonal patterns
ROLLS = [3, 6, 12]    # Short/medium/long rolling statistics
QUANTILES = [0.1, 0.5, 0.9]  # Explore risk quantiles

# Feature engineering: construct lag + rolling statistics
df = dfs[DATASET].copy()
df["timestamp"] = pd.to_datetime(df["timestamp"]).dt.tz_localize(None)
df = df.sort_values(["series_name", "timestamp"])
df["y"] = pd.to_numeric(df["y"], errors="coerce")

for lag in LAGS:
    # Lagged values: capture autoregressive structure
    df[f"lag_{lag}"] = df.groupby("series_name")["y"].shift(lag)

for r in ROLLS:
    # Ensure rolling features use ONLY past data (shift(1) prevents leakage)
    df[f"roll_mean_{r}"] = df.groupby("series_name")["y"].shift(1).rolling(r).mean()
    df[f"roll_std_{r}"]  = df.groupby("series_name")["y"].shift(1).rolling(r).std()

feature_cols = [c for c in df.columns if c.startswith(("lag_", "roll_"))]

def build_global_sets(horizon):
    """
    Construct global train/test matrices by pooling all series for a given horizon.
    This leverages cross-series information: large hospitals inform predictions of small ones.
    """
    X_train, y_train = [], []
    X_test, y_test = [], []

    splits = splits_df[
        (splits_df["dataset"] == DATASET) &
        (splits_df["horizon"] == horizon)
    ]

    for _, s in splits.iterrows():
        g = df[df["series_name"] == s["series_name"]]

        train = g[g["timestamp"] <= s["train_end"]].dropna(subset=feature_cols)
        test  = g[(g["timestamp"] >= s["test_start"]) &
                  (g["timestamp"] <= s["test_end"])].dropna(subset=feature_cols)

        if len(train) == 0 or len(test) == 0:
            continue

        X_train.append(train[feature_cols])
        y_train.append(train["y"])
        X_test.append(test[feature_cols])
        y_test.append(test["y"])

    return (
        pd.concat(X_train, ignore_index=True),
        pd.concat(y_train, ignore_index=True),
        pd.concat(X_test, ignore_index=True),
        pd.concat(y_test, ignore_index=True)
    )

# Train global median (α=0.5) model only for speed
results = []

for horizon in HORIZONS:
    Xtr, ytr, Xte, yte = build_global_sets(horizon)
    print(f"\nHorizon {horizon}: train={Xtr.shape}, test={Xte.shape}")

    # Quantile regression: α=0.5 ≈ median; robust to outliers
    model = lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.5,
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )

    model.fit(Xtr, ytr)
    preds = model.predict(Xte)

    mae = mean_absolute_error(yte, preds)
    mse = mean_squared_error(yte, preds)
    rmse = float(np.sqrt(mse))
    results.append({
        "dataset": DATASET,
        "horizon": horizon,
        "model": "LightGBM (global, unfiltered)",
        "MAE": float(mae),
        "RMSE": rmse
    })

results_df = pd.DataFrame(results)

print("\n" + "="*90)
print("HOSPITAL ML — RAPID PROTOTYPE (GLOBAL, UNCONSTRAINED)")
print("="*90)
print(results_df)


Horizon 6: train=(430287, 10), test=(50622, 10)

Horizon 12: train=(331344, 10), test=(82836, 10)

HOSPITAL ML — RAPID PROTOTYPE (GLOBAL, UNCONSTRAINED)
                dataset  horizon                          model        MAE  \
0  hospital_dataset.tsf        6  LightGBM (global, unfiltered)  17.674858   
1  hospital_dataset.tsf       12  LightGBM (global, unfiltered)  18.052381   

        RMSE  
0  57.376329  
1  59.459905  


## Cell 10a: Distribution Analysis & Error Characterization

### Why Analyze Residuals?
Forecasting is fundamentally about understanding uncertainty:
- **Tail behavior** (p90, p95, p99): identifies outliers and model stress points
- **Absolute error distribution**: tells us whether MAE is driven by typical errors or rare blowups
- **Horizon effect**: errors often grow linearly or super-linearly with forecast horizon

This analysis shapes downstream decisions: segmentation by volume is only worth it if global model errors correlate with series scale.

In [7]:
import numpy as np
import pandas as pd

def build_global_sets_only_test(horizon):
    """Rebuild test set for residual analysis (same features as training)."""
    X_test, y_test = [], []

    splits = splits_df[
        (splits_df["dataset"] == "hospital_dataset.tsf") &
        (splits_df["horizon"] == horizon)
    ]

    for _, s in splits.iterrows():
        g = df[df["series_name"] == s["series_name"]]
        test  = g[(g["timestamp"] >= s["test_start"]) &
                  (g["timestamp"] <= s["test_end"])].dropna(subset=feature_cols)
        if len(test) == 0:
            continue
        X_test.append(test[feature_cols])
        y_test.append(test["y"])

    return pd.concat(X_test, ignore_index=True), pd.concat(y_test, ignore_index=True)

def quick_stats(arr, name):
    """Compute percentile summary to identify tail behavior."""
    arr = np.asarray(arr, dtype=float)
    print(f"\n{name}")
    print(f"  count: {len(arr)}")
    print(f"  min/median/max: {np.min(arr):.3f} / {np.median(arr):.3f} / {np.max(arr):.3f}")
    print(f"  p90/p95/p99: {np.percentile(arr, 90):.3f} / {np.percentile(arr, 95):.3f} / {np.percentile(arr, 99):.3f}")

import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error

for horizon in [6, 12]:
    print(f"\n{'='*90}")
    print(f"RESIDUAL ANALYSIS FOR HORIZON {horizon}")
    print(f"{'='*90}")
    
    Xte, yte = build_global_sets_only_test(horizon)
    quick_stats(yte.values, f"y_true distribution (h={horizon})")

    # Retrain model to obtain residuals
    Xtr, ytr, _, _ = build_global_sets(horizon)

    model = lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.5,
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )
    model.fit(Xtr, ytr)
    preds = model.predict(Xte)

    quick_stats(preds, f"y_pred distribution (h={horizon})")

    abs_err = np.abs(yte.values - preds)
    quick_stats(abs_err, f"|error| distribution (h={horizon})")

    mae = mean_absolute_error(yte, preds)
    rmse = np.sqrt(mean_squared_error(yte, preds))
    print(f"\n  Aggregate: MAE={mae:.3f}  RMSE={rmse:.3f}")

    # Identify worst-performing forecasts for debugging
    worst_idx = np.argsort(abs_err)[-10:][::-1]
    worst_df = pd.DataFrame({
        "y_true": yte.values[worst_idx],
        "y_pred": preds[worst_idx],
        "abs_err": abs_err[worst_idx]
    })
    print(f"\n  Top 10 worst errors (debugging artifacts):")
    print(worst_df.to_string())


RESIDUAL ANALYSIS FOR HORIZON 6

y_true distribution (h=6)
  count: 50622
  min/median/max: 1.000 / 40.000 / 12090.000
  p90/p95/p99: 659.000 / 1253.000 / 3862.000

y_pred distribution (h=6)
  count: 50622
  min/median/max: 5.448 / 40.062 / 11060.372
  p90/p95/p99: 663.967 / 1234.312 / 3916.483

|error| distribution (h=6)
  count: 50622
  min/median/max: 0.000 / 5.639 / 1590.878
  p90/p95/p99: 35.865 / 65.150 / 206.426

  Aggregate: MAE=17.675  RMSE=57.376

  Top 10 worst errors (debugging artifacts):
   y_true        y_pred      abs_err
0    9407   7816.122077  1590.877923
1    9407   7816.122077  1590.877923
2    8899   7311.957456  1587.042544
3    8899   7311.957456  1587.042544
4    6846   8337.593563  1491.593563
5   10154   8765.282527  1388.717473
6   12090  10867.136985  1222.863015
7   12090  10867.136985  1222.863015
8    6357   7487.833060  1130.833060
9    6357   7487.833060  1130.833060

RESIDUAL ANALYSIS FOR HORIZON 12

y_true distribution (h=12)
  count: 82836
  min/me

## Cell 10b: Log-Transform for Variance Stabilization

### The Heavy-Tail Problem
Hospital demand data often exhibits **multiplicative noise** rather than additive:
- Large hospitals (high volume) have larger absolute errors (high variance)
- Small hospitals (low volume) have tight control (low variance)
- **Coefficient of variation** (CV = σ/μ) is roughly constant across scales

### Why log1p Works
Taking log(1 + y) stabilizes the variance and transforms the problem to additive noise:
- Prevents model from over-fitting to large facilities
- Enables cross-series transfer learning (global model sees similar scaled inputs)
- Inverse transform via expm1 recovers original scale without bias

**Note**: log1p handles y=0 gracefully (log(1+0)=0); pure log would fail.

In [8]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error

DATASET = "hospital_dataset.tsf"
HORIZONS = [6, 12]

results = []

for horizon in HORIZONS:
    Xtr, ytr, Xte, yte = build_global_sets(horizon)

    # Apply log1p to stabilize variance across heterogeneous scales
    ytr_log = np.log1p(ytr)

    print(f"\nHorizon {horizon}: train={Xtr.shape}, test={Xte.shape}")
    print(f"  Original y: μ={ytr.mean():.1f}, σ={ytr.std():.1f}")
    print(f"  Log-transformed y: μ={ytr_log.mean():.3f}, σ={ytr_log.std():.3f}")

    model = lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.5,
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )

    model.fit(Xtr, ytr_log)

    # Predict in log space, then invert to original scale
    preds_log = model.predict(Xte)
    preds = np.expm1(preds_log)

    mae = mean_absolute_error(yte, preds)
    rmse = np.sqrt(mean_squared_error(yte, preds))

    results.append({
        "dataset": DATASET,
        "horizon": horizon,
        "model": "LightGBM (global, log1p target)",
        "MAE": float(mae),
        "RMSE": float(rmse)
    })

results_df = pd.DataFrame(results)

print("\n" + "="*90)
print("HOSPITAL ML — LOG1P VARIANCE STABILIZATION")
print("="*90)
print(results_df)


Horizon 6: train=(430287, 10), test=(50622, 10)
  Original y: μ=266.2, σ=830.9
  Log-transformed y: μ=4.087, σ=1.517

Horizon 12: train=(331344, 10), test=(82836, 10)
  Original y: μ=265.5, σ=828.6
  Log-transformed y: μ=4.083, σ=1.518

HOSPITAL ML — LOG1P VARIANCE STABILIZATION
                dataset  horizon                            model        MAE  \
0  hospital_dataset.tsf        6  LightGBM (global, log1p target)  17.340464   
1  hospital_dataset.tsf       12  LightGBM (global, log1p target)  17.978900   

        RMSE  
0  55.842261  
1  60.299794  


## Cell 11: Segmented Global LightGBM — Training Median Bucketing

### Core Design: Why Segment?
The bias-variance tradeoff is fundamental here:
- **Without segmentation**: global model must interpolate across extreme scales (high variance)
- **With segmentation**: each model sees homogeneous ranges (lower variance per bucket)
- **Cost**: slightly higher bias (less data per model), but net MSE often improves

### Bucketing Strategy
We partition series by **training-period median volume** into tertiles (small/medium/large):
- Uses ONLY training history (no leakage from test set)
- Ensures each bucket has similar forecast scales
- Stable across rolling windows (cutoff at minimum train_end date)

### Implementation: log1p + Feature Lag-Shift
- Apply log transform per bucket to further stabilize variance
- Shift rolling features by 1 period to prevent accidental leakage

In [9]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error

DATASET = "hospital_dataset.tsf"
HORIZONS = [6, 12]

LAGS  = [1, 3, 6, 12]
ROLLS = [3, 6, 12]

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# Prepare data with timezone consistency
df = dfs[DATASET].copy()
df["timestamp"] = pd.to_datetime(df["timestamp"]).dt.tz_localize(None)
df["y"] = pd.to_numeric(df["y"], errors="coerce")
df = df.sort_values(["series_name", "timestamp"]).reset_index(drop=True)

spl = splits_df[splits_df["dataset"] == DATASET].copy()
for c in ["train_end", "test_start", "test_end"]:
    spl[c] = pd.to_datetime(spl[c]).dt.tz_localize(None)

# Construct features (guaranteed)
for lag in LAGS:
    df[f"lag_{lag}"] = df.groupby("series_name")["y"].shift(lag)

for r in ROLLS:
    # Shift(1) ensures rolling uses only past observations
    df[f"roll_mean_{r}"] = (
        df.groupby("series_name")["y"]
          .shift(1)
          .rolling(r)
          .mean()
          .reset_index(level=0, drop=True)
    )
    df[f"roll_std_{r}"] = (
        df.groupby("series_name")["y"]
          .shift(1)
          .rolling(r)
          .std()
          .reset_index(level=0, drop=True)
    )

feature_cols = [c for c in df.columns if c.startswith(("lag_", "roll_"))]

# Segment series by training-period median volume (no test-set leakage)
train_cutoff = spl["train_end"].min()
train_hist = df[df["timestamp"] <= train_cutoff]

series_median = train_hist.groupby("series_name")["y"].median()
q1, q2 = series_median.quantile([0.33, 0.66])

def size_bucket(v):
    if v <= q1:
        return "small"
    elif v <= q2:
        return "medium"
    else:
        return "large"

series_bucket = series_median.apply(size_bucket)
df["bucket"] = df["series_name"].map(series_bucket)

print(f"Bucket thresholds: q1={q1:.1f}, q2={q2:.1f}")
print(f"Series per bucket: {series_bucket.value_counts().to_dict()}")

def build_bucket_sets(horizon, bucket):
    """Construct global train/test matrices for a specific size bucket."""
    Xtr, ytr, Xte, yte = [], [], [], []

    splits_h = spl[spl["horizon"] == horizon]
    for _, s in splits_h.iterrows():
        g = df[(df["series_name"] == s["series_name"]) & (df["bucket"] == bucket)]
        if g.empty:
            continue

        train = g[g["timestamp"] <= s["train_end"]].dropna(subset=feature_cols)
        test  = g[(g["timestamp"] >= s["test_start"]) &
                  (g["timestamp"] <= s["test_end"])].dropna(subset=feature_cols)

        if len(train) == 0 or len(test) == 0:
            continue

        Xtr.append(train[feature_cols]); ytr.append(train["y"])
        Xte.append(test[feature_cols]);  yte.append(test["y"])

    if not Xtr:
        return None

    return (
        pd.concat(Xtr, ignore_index=True),
        pd.concat(ytr, ignore_index=True),
        pd.concat(Xte, ignore_index=True),
        pd.concat(yte, ignore_index=True),
    )

# Train 1 model per (horizon, bucket) with log1p variance stabilization
bucket_metrics = []
bucket_counts = []
models = {}  # Store fitted models for SHAP

for horizon in HORIZONS:
    for bucket in ["small", "medium", "large"]:
        data = build_bucket_sets(horizon, bucket)
        if data is None:
            print(f"H{horizon} | {bucket:<6} | SKIP (no data)")
            continue

        Xtr, ytr, Xte, yte = data
        n_test = len(yte)

        print(f"H{horizon} | {bucket:<6} | train={Xtr.shape} test={Xte.shape}")

        # Apply log transform to handle heavy tails within each bucket
        ytr_log = np.log1p(ytr)

        model = lgb.LGBMRegressor(
            objective="quantile",
            alpha=0.5,
            n_estimators=300,
            learning_rate=0.05,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            verbosity=-1
        )
        model.fit(Xtr, ytr_log)
        models[(horizon, bucket)] = model

        # Predict and invert back to original scale
        preds = np.expm1(model.predict(Xte))

        mae_val = float(mean_absolute_error(yte, preds))
        rmse_val = rmse(yte, preds)

        bucket_metrics.append({
            "dataset": DATASET,
            "horizon": horizon,
            "bucket": bucket,
            "model": "LightGBM (segmented, log1p)",
            "MAE": mae_val,
            "RMSE": rmse_val
        })
        bucket_counts.append({
            "horizon": horizon,
            "bucket": bucket,
            "n_test": int(n_test)
        })

bucket_df = pd.DataFrame(bucket_metrics).sort_values(["horizon", "bucket"]).reset_index(drop=True)
counts_df = pd.DataFrame(bucket_counts)

print("\n" + "="*100)
print("SEGMENTED GLOBAL LIGHTGBM — BUCKET METRICS")
print("="*100)
print(bucket_df)

# Compute weighted aggregate metrics
weighted_rows = []
for horizon in HORIZONS:
    sub = bucket_df[bucket_df["horizon"] == horizon].merge(
        counts_df[counts_df["horizon"] == horizon],
        on=["horizon", "bucket"],
        how="inner"
    )
    total_n = sub["n_test"].sum()

    w_mae = (sub["MAE"] * sub["n_test"]).sum() / total_n
    w_rmse = float(np.sqrt(((sub["RMSE"] ** 2) * sub["n_test"]).sum() / total_n))

    weighted_rows.append({
        "dataset": DATASET,
        "horizon": horizon,
        "model": "LightGBM (segmented, weighted)",
        "Weighted_MAE": float(w_mae),
        "Weighted_RMSE": float(w_rmse),
        "Total_Test_Rows": int(total_n)
    })

weighted_df = pd.DataFrame(weighted_rows).sort_values("horizon").reset_index(drop=True)

print("\n" + "="*100)
print("SEGMENTED GLOBAL LIGHTGBM — WEIGHTED OVERALL METRICS")
print("="*100)
print(weighted_df)

Bucket thresholds: q1=22.0, q2=73.6
Series per bucket: {'small': 267, 'large': 261, 'medium': 239}
H6 | small  | train=(149787, 10) test=(17622, 10)
H6 | medium | train=(134079, 10) test=(15774, 10)
H6 | large  | train=(146421, 10) test=(17226, 10)
H12 | small  | train=(115344, 10) test=(28836, 10)
H12 | medium | train=(103248, 10) test=(25812, 10)
H12 | large  | train=(112752, 10) test=(28188, 10)

SEGMENTED GLOBAL LIGHTGBM — BUCKET METRICS
                dataset  horizon  bucket                        model  \
0  hospital_dataset.tsf        6   large  LightGBM (segmented, log1p)   
1  hospital_dataset.tsf        6  medium  LightGBM (segmented, log1p)   
2  hospital_dataset.tsf        6   small  LightGBM (segmented, log1p)   
3  hospital_dataset.tsf       12   large  LightGBM (segmented, log1p)   
4  hospital_dataset.tsf       12  medium  LightGBM (segmented, log1p)   
5  hospital_dataset.tsf       12   small  LightGBM (segmented, log1p)   

         MAE       RMSE  
0  38.948141  89

## Cell 13: SHAP-Based Feature Importance with Additivity Override

### Why SHAP?
Tree-based models (LightGBM) are black-box by nature. SHAP (SHapley Additive exPlanations) provides:
- **Local Interpretability**: explain each individual forecast
- **Feature Ranking**: identify which features drive predictions most
- **Theoretically Sound**: based on cooperative game theory (Shapley values)

### The Additivity Challenge
LightGBM's TreeExplainer has a built-in consistency check (`check_additivity=True` by default). However:
- **High-dimensional interactions**: when features interact strongly (e.g., lag_1 × roll_mean_3), the check may fail
- **Solution**: `check_additivity=False` disables the check, allowing feature_perturbation="interventional" to compute SHAP values robustly
- **Downside**: you lose the built-in guardrail; interpretation requires more care

### Workflow
1. Sample background data (2000 rows) for efficiency
2. Create explainer with `feature_perturbation="interventional"` (respects feature correlations)
3. Compute SHAP values for up to 10,000 test samples
4. Average absolute SHAP values per feature to rank importance

In [10]:
import os
import numpy as np
import pandas as pd

# Verify required variables exist from segmented training cell
req = ["df", "spl", "feature_cols", "models", "DATASET", "HORIZONS"]
missing = [r for r in req if r not in globals()]
if missing:
    raise RuntimeError(
        f"Missing required variables: {missing}\n"
        "Run the segmented LightGBM training cell first."
    )

OUT_DIR = "./artifacts_interpretability"
os.makedirs(OUT_DIR, exist_ok=True)

MAX_BACKGROUND = 2000
MAX_EXPLAIN    = 10000
RANDOM_SEED    = 42
rng = np.random.default_rng(RANDOM_SEED)

def build_bucket_test_matrix(horizon: int, bucket: str):
    """Construct test feature matrix for a given (horizon, bucket) pair."""
    Xte_list, yte_list = [], []
    splits_h = spl[spl["horizon"] == horizon]

    for _, s in splits_h.iterrows():
        g = df[(df["series_name"] == s["series_name"]) & (df["bucket"] == bucket)]
        if g.empty:
            continue

        test = g[(g["timestamp"] >= s["test_start"]) & (g["timestamp"] <= s["test_end"])].dropna(subset=feature_cols)
        if len(test) == 0:
            continue

        Xte_list.append(test[feature_cols])
        yte_list.append(test["y"])

    if not Xte_list:
        return None

    Xte = pd.concat(Xte_list, ignore_index=True)
    yte = pd.concat(yte_list, ignore_index=True)
    return Xte, yte

def sample_df(X: pd.DataFrame, max_rows: int):
    """Randomly sample DataFrame rows for computational efficiency."""
    if len(X) <= max_rows:
        return X.reset_index(drop=True)
    idx = rng.choice(len(X), size=max_rows, replace=False)
    return X.iloc[idx].reset_index(drop=True)

# Try SHAP; fallback to permutation importance if unavailable
use_shap = True
try:
    import shap
except Exception:
    use_shap = False

all_rows = []

if use_shap:
    print("✓ SHAP available. Using TreeExplainer with check_additivity=False.")
    print("  (Disables additivity check to handle high-dimensional interaction effects.)\n")
    
    for horizon in HORIZONS:
        for bucket in ["small", "medium", "large"]:
            key = (horizon, bucket)
            if key not in models:
                print(f"  SKIP: no model for H{horizon} {bucket}")
                continue

            data = build_bucket_test_matrix(horizon, bucket)
            if data is None:
                print(f"  SKIP: no test data for H{horizon} {bucket}")
                continue

            Xte, _ = data

            # Sample for efficiency
            X_bg = sample_df(Xte, MAX_BACKGROUND)
            X_ex = sample_df(Xte, MAX_EXPLAIN)

            model = models[key]

            # Create explainer with feature_perturbation="interventional" (respects correlations)
            explainer = shap.TreeExplainer(
                model, 
                data=X_bg, 
                feature_perturbation="interventional"
            )
            
            # CRITICAL: disable additivity check to allow high-dimensional interactions
            shap_vals = explainer.shap_values(X_ex, check_additivity=False)

            # Average absolute SHAP values across samples to rank features
            imp = np.mean(np.abs(shap_vals), axis=0)
            imp_df = pd.DataFrame({
                "feature": feature_cols, 
                "mean_abs_shap": imp
            })
            imp_df = imp_df.sort_values("mean_abs_shap", ascending=False).reset_index(drop=True)

            imp_df["dataset"] = DATASET
            imp_df["horizon"] = horizon
            imp_df["bucket"] = bucket
            all_rows.append(imp_df)

            out_path = os.path.join(OUT_DIR, f"shap_importance_{DATASET}_H{horizon}_{bucket}.csv")
            imp_df.to_csv(out_path, index=False)
            print(f"  ✓ Saved {out_path} | explained={len(X_ex)} | bg={len(X_bg)}")

    if not all_rows:
        raise RuntimeError("No SHAP outputs produced; check models and data.")

    shap_all = pd.concat(all_rows, ignore_index=True)

    TOPK = 15
    topk = (
        shap_all
        .sort_values(["horizon", "bucket", "mean_abs_shap"], ascending=[True, True, False])
        .groupby(["dataset", "horizon", "bucket"], as_index=False)
        .head(TOPK)
        .reset_index(drop=True)
    )

    print("\n" + "="*100)
    print("TOP SHAP FEATURES (mean |SHAP|) — per horizon & bucket")
    print("="*100)
    print(topk)

    topk_path = os.path.join(OUT_DIR, f"shap_top{TOPK}_{DATASET}.csv")
    topk.to_csv(topk_path, index=False)
    print(f"\n✓ Saved: {topk_path}")

else:
    print("⚠ SHAP not available; using permutation importance as fallback.")

c:\Users\aarav\anaconda3\envs\mp3env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ SHAP available. Using TreeExplainer with check_additivity=False.
  (Disables additivity check to handle high-dimensional interaction effects.)



 99%|===================| 9933/10000 [00:51<00:00]        

  ✓ Saved ./artifacts_interpretability\shap_importance_hospital_dataset.tsf_H6_small.csv | explained=10000 | bg=2000


 99%|===================| 9931/10000 [00:55<00:00]        

...

  ✓ Saved ./artifacts_interpretability\shap_importance_hospital_dataset.tsf_H12_small.csv | explained=10000 | bg=2000


100%|===================| 9991/10000 [00:56<00:00]        

  ✓ Saved ./artifacts_interpretability\shap_importance_hospital_dataset.tsf_H12_medium.csv | explained=10000 | bg=2000


100%|===================| 9982/10000 [00:56<00:00]        

  ✓ Saved ./artifacts_interpretability\shap_importance_hospital_dataset.tsf_H12_large.csv | explained=10000 | bg=2000

TOP SHAP FEATURES (mean |SHAP|) — per horizon & bucket
         feature  mean_abs_shap               dataset  horizon  bucket
0          lag_1       0.367885  hospital_dataset.tsf        6   large
1    roll_mean_3       0.305724  hospital_dataset.tsf        6   large
2         lag_12       0.111043  hospital_dataset.tsf        6   large
3   roll_mean_12       0.105898  hospital_dataset.tsf        6   large
4    roll_mean_6       0.051217  hospital_dataset.tsf        6   large
5          lag_3       0.015017  hospital_dataset.tsf        6   large
6          lag_6       0.013568  hospital_dataset.tsf        6   large
7    roll_std_12       0.004639  hospital_dataset.tsf        6   large
8     roll_std_6       0.004469  hospital_dataset.tsf        6   large
9     roll_std_3       0.004287  hospital_dataset.tsf        6   large
10   roll_mean_3       0.131671  hospital_dat

## Cell 14: LLM-Ready Artifact Packaging

### Packaging Strategy
We export model outputs and interpretability results to a structured JSON + CSV format ready for downstream LLM narration:
- **Single JSON "packet"**: high-level metrics + top features (compact, LangGraph-friendly)
- **Individual CSVs**: detailed per-split/per-bucket breakdowns (queryable by retrieval systems)
- **Prompt templates**: system/user instructions for DeepSeek or other LLMs

### Why This Separation?
- LangGraph pipelines process JSON efficiently (no dataframe overhead)
- RAG systems can retrieve detailed CSVs on demand
- Prevents hallucination: LLM only sees data that actually exists in the outputs

In [11]:
import os
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd

OUT_DIR = Path("./LLM_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def _safe_write_df(df: pd.DataFrame, filename: str):
    """Write DataFrame to CSV, skip if None or empty."""
    if df is None or not isinstance(df, pd.DataFrame) or df.empty:
        return
    out_path = OUT_DIR / filename
    df.to_csv(out_path, index=False)
    print(f"[saved] {out_path}  rows={len(df):,}  cols={df.shape[1]}")

def _safe_write_json(obj, filename: str):
    """Write JSON object, converting non-serializable types."""
    out_path = OUT_DIR / filename
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, default=str)
    print(f"[saved] {out_path}")

def _maybe_get_df(name: str):
    """Safely retrieve DataFrame from namespace."""
    return globals().get(name, None)

# Export core metric tables
_safe_write_df(_maybe_get_df("baseline_df"), "baseline_seasonal_naive_rows.csv")
_safe_write_df(_maybe_get_df("bucket_df"), "lgbm_segmented_bucket_metrics.csv")
_safe_write_df(_maybe_get_df("weighted_df"), "lgbm_segmented_weighted_metrics.csv")

# Aggregate and export SHAP summaries
ART_DIR = Path("./artifacts_interpretability")
shap_top_files = sorted(ART_DIR.glob("shap_top*_*.csv")) if ART_DIR.exists() else []
shap_imp_files = sorted(ART_DIR.glob("shap_importance_*.csv")) if ART_DIR.exists() else []

def _concat_csvs(files):
    """Concatenate multiple CSV files, skip on read failure."""
    if not files:
        return None
    dfs_ = []
    for fp in files:
        try:
            df_ = pd.read_csv(fp)
            df_["source_file"] = fp.name
            dfs_.append(df_)
        except Exception as e:
            print(f"  [warn] failed reading {fp}: {e}")
    if not dfs_:
        return None
    return pd.concat(dfs_, ignore_index=True)

shap_imp_all = _concat_csvs(shap_imp_files)
_safe_write_df(shap_imp_all, "interpret_shap_importance_all.csv")

# Construct LLM packet: compact JSON with all key results
packet = {}

def _df_to_records(df):
    """Convert DataFrame to list of dicts, replace NaN with None for JSON."""
    if df is None or not isinstance(df, pd.DataFrame) or df.empty:
        return None
    return df.replace({np.nan: None}).to_dict(orient="records")

# Package metrics
packet["metrics"] = {
    "baseline_summary": _df_to_records(_maybe_get_df("summary")) if isinstance(_maybe_get_df("summary"), pd.DataFrame) else None,
    "lgbm_segmented_bucket_metrics": _df_to_records(_maybe_get_df("bucket_df")),
    "lgbm_segmented_weighted_metrics": _df_to_records(_maybe_get_df("weighted_df")),
}

# Package interpretability (top features only, not full importance)
if shap_imp_all is not None and not shap_imp_all.empty:
    top_feat = shap_imp_all.nlargest(15, "mean_abs_shap")  # Top 15 per bucket
    packet["interpretability"] = {
        "method": "shap",
        "top_features": _df_to_records(top_feat)
    }
else:
    packet["interpretability"] = {"method": None, "top_features": None}

_safe_write_json(packet, "llm_packet.json")

# Write prompt templates (instructions for LLM)
SYSTEM_PROMPT = """You are an analytics assistant interpreting hospital forecasting results.
You must ONLY cite metrics and features present in the provided data.
You must NOT forecast new values, hallucinate numbers, or invent insights.

Output structure:
1) Executive Summary: 3-5 key findings (MAE/RMSE improvements, bucket effects)
2) Model Comparison: Baseline vs. ML across horizons
3) Segment Insights: Small/Medium/Large hospital differences
4) Top Drivers: Which features matter most (from SHAP)
5) Recommendations: Deployment guidance based on data
"""

USER_PROMPT = """Interpret the attached hospital forecasting outputs.

Use ONLY the provided CSVs and JSON:
- lgbm_segmented_weighted_metrics.csv: overall model performance
- lgbm_segmented_bucket_metrics.csv: per-segment breakdown
- baseline_seasonal_naive_rows.csv: baseline benchmark
- interpret_shap_importance_all.csv: feature importance
- llm_packet.json: executive summary packet

Rules:
- Do not invent metrics.
- Do not hallucinate model details.
- Explain WHY segmentation works (if it does) using the provided metrics.
"""

(OUT_DIR / "prompt_system.txt").write_text(SYSTEM_PROMPT, encoding="utf-8")
(OUT_DIR / "prompt_user.txt").write_text(USER_PROMPT, encoding="utf-8")
print(f"[saved] {OUT_DIR / 'prompt_system.txt'}")
print(f"[saved] {OUT_DIR / 'prompt_user.txt'}")

print("\n✓ LLM-ready artifacts packaged. Ready for downstream narration pipeline.")

[saved] LLM_outputs\baseline_seasonal_naive_rows.csv  rows=30,358  cols=6
[saved] LLM_outputs\lgbm_segmented_bucket_metrics.csv  rows=6  cols=6
[saved] LLM_outputs\lgbm_segmented_weighted_metrics.csv  rows=2  cols=6
[saved] LLM_outputs\interpret_shap_importance_all.csv  rows=60  cols=6
[saved] LLM_outputs\llm_packet.json
[saved] LLM_outputs\prompt_system.txt
[saved] LLM_outputs\prompt_user.txt

✓ LLM-ready artifacts packaged. Ready for downstream narration pipeline.


## Cell 15: LLM Input Preparation & DeepSeek Integration

### Purpose
This cell prepares a minimal context JSON from all computed metrics and feeds it to DeepSeek API for automated narrative generation.

### Design Philosophy
- **Constraints-Based**: LLM sees only facts (CSV data + JSON), no template boilerplate
- **Fact-Checking Ready**: if LLM cites a metric, it's verifiable against source CSVs
- **Deployable**: can be wrapped in LangGraph for multi-step reasoning

In [12]:
import os
import json
import pandas as pd
from datetime import datetime

INPUT_DIR = "./LLM_input"
os.makedirs(INPUT_DIR, exist_ok=True)

def safe_save(df, filename):
    """Save DataFrame to CSV for LLM consumption."""
    if df is not None and isinstance(df, pd.DataFrame) and not df.empty:
        path = os.path.join(INPUT_DIR, filename)
        df.to_csv(path, index=False)
        print(f"[Saved] {filename}")
        return True
    return False

print("PREPARING CONTEXT FOR LLM NARRATION...\n")

# Save metric tables
safe_save(globals().get('baseline_df'), "baseline_metrics.csv")
safe_save(globals().get('bucket_df'), "ml_segmented_buckets.csv")
safe_save(globals().get('weighted_df'), "ml_segmented_weighted.csv")
safe_save(globals().get('topk'), "shap_top_drivers.csv")

# Metadata
context_note = {
    "project_scope": "Hospital Capacity Planning (Time Series Forecasting)",
    "dataset": "hospital_dataset.tsf",
    "horizons_months": [6, 12],
    "constraints": [
        "Only cite provided metrics; do not forecast new values.",
        "Compare models on MAE (robustness) and RMSE (outlier penalty).",
        "Explain segmentation benefits using provided metrics."
    ],
    "generated_at": datetime.now().isoformat()
}

with open(os.path.join(INPUT_DIR, "context.json"), "w") as f:
    json.dump(context_note, f, indent=2)

print(f"\n✓ Context prepared in {INPUT_DIR}")

PREPARING CONTEXT FOR LLM NARRATION...

[Saved] baseline_metrics.csv
[Saved] ml_segmented_buckets.csv
[Saved] ml_segmented_weighted.csv
[Saved] shap_top_drivers.csv

✓ Context prepared in ./LLM_input


## Cell 16: Automated Narrative Generation via DeepSeek API

### Workflow
1. **Load Metrics**: read CSV files and context JSON
2. **Construct Prompt**: system (role) + user (task) combine to guide narration
3. **Call DeepSeek**: remote API generates structured report
4. **Persist Output**: save narrative for stakeholder review

### Key Principle
The LLM acts as a **lazy summarizer**, not a researcher. It does not:
- Retrain models
- Compute new metrics
- Invent thresholds

Instead, it:
- Interprets provided metrics
- Connects findings to hospital domain (e.g., "large facilities need 12-month forecasts")
- Recommends based on observed performance

In [ ]:
import os
import json
import pandas as pd
from openai import OpenAI

# ---------------------------------------------------------
# CONFIG: API & OUTPUT
# ---------------------------------------------------------
DEEPSEEK_API_KEY = "add any openAI/deepseek api key here" 

OUTPUT_DIR = "./Deepseek_results"
INPUT_DIR = "./LLM_input"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------------------------------------------------
# 1. RAW CONTENT LOADER (WITH TOKEN SAFETY)
# ---------------------------------------------------------
def get_raw_content(filename, max_rows=200):
    path = os.path.join(INPUT_DIR, filename)
    if not os.path.exists(path):
        return "Data not found."
    
    try:
        df = pd.read_csv(path)
        
        # CASE A: Small, high-value files (SHAP, Segmented Metrics, ETS)
        # We want the LLM to see EVERY row here, especially for SHAP feature names.
        if len(df) < max_rows:
            return df.to_string(index=False)
        
        # CASE B: Massive files (Row-level Baseline/Global backtests)
        # We cannot send 30k rows (800k tokens). We sample the head/tail to show structure.
        else:
            return (
                f"Note: File has {len(df)} rows. Showing first {max_rows} rows only:\n" +
                df.head(max_rows).to_string(index=False) + 
                "\n... [TRUNCATED DUE TO CONTEXT LIMIT] ..."
            )
    except Exception as e:
        return f"Error reading file: {str(e)}"

# Load raw strings. Note we don't use 'describe()' anymore.
context_str = f"""
PROJECT METADATA:
{get_raw_content('context.json')}

MODEL PERFORMANCE (RAW DATA):
- Baseline (Seasonal Naive) [Sampled]: 
{get_raw_content('baseline_metrics.csv', max_rows=50)}

- Classical (ETS): 
{get_raw_content('ets_metrics.csv')}

- Global ML Metrics: 
{get_raw_content('ml_global_metrics.csv')}

- Segmented ML (Buckets) [CRITICAL]: 
{get_raw_content('ml_segmented_buckets.csv')}

- Segmented ML (Weighted Overall): 
{get_raw_content('ml_segmented_weighted.csv')}

INTERPRETABILITY (SHAP TOP DRIVERS) [FULL LIST]:
{get_raw_content('shap_top_drivers.csv', max_rows=1000)} 
"""

# ---------------------------------------------------------
# 2. CONSTRUCT PROMPT
# ---------------------------------------------------------
# We emphasize to the model that it is looking at raw data segments.
system_prompt = """You are a Principal Data Scientist. 
You are analyzing the RAW output metrics from a Hospital Forecasting Pipeline.

OBJECTIVES:
1. Compare Classical ETS vs. Segmented ML. 
2. Diagnose the specific failure in the 'Large' hospital bucket using the raw metrics.
3. Look at the SHAP table to identify WHICH features (lag vs roll) drive specific segments.

OUTPUT STRUCTURE:
- Executive Summary
- Leaderboard (Rank models by robustness)
- Deep Dive: The Large Hospital Failure Mode (Cite specific RMSE/MAE values)
- Driver Analysis: Mapping feature importance to hospital scale
- Final Recommendation
"""

user_prompt = f"Analyze these raw results:\n\n{context_str}"

# ---------------------------------------------------------
# 3. CALL DEEPSEEK REASONER
# ---------------------------------------------------------
print("⏳ Contacting DeepSeek Reasoner (Thinking Model)...")

client = OpenAI(
    api_key=DEEPSEEK_API_KEY, 
    base_url="https://api.deepseek.com"
)

try:
    response = client.chat.completions.create(
        model="deepseek-reasoner", 
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    narrative = response.choices[0].message.content
    
    out_path = os.path.join(OUTPUT_DIR, "hospital_thinking_narrative_raw.md")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(narrative)
    
    print("-" * 80)
    print(narrative)
    print("-" * 80)
    print(f"\n✅ Thinking report saved to: {out_path}")

except Exception as e:
    print(f"\n❌ API Error: {e}")

⏳ Contacting DeepSeek Reasoner (Thinking Model)...
--------------------------------------------------------------------------------
# Hospital Forecasting Pipeline Analysis

## Executive Summary

The analysis reveals a **critical failure mode in the "Large" hospital segment** that dramatically degrades overall performance of the Segmented ML approach. While Classical ETS provides consistent moderate performance across all hospital sizes, the Segmented ML model catastrophically fails for large hospitals despite strong performance on small and medium hospitals. The Large bucket's RMSE (89.26) is **4-5x worse** than Classical ETS (21.78), indicating a fundamental mismatch between the ML model and large-hospital patterns.

## Leaderboard: Ranked by Robustness

### 1. **Classical ETS** (Most Robust)
- **Overall MAE**: 18.59-20.49 | **Overall RMSE**: 21.78-24.42
- **Strengths**: Consistent performance across all hospital scales, no catastrophic failures
- **Weaknesses**: Moderate accuracy, n